In [1]:
import torch
import pickle
import numpy as np
import os.path as osp
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset, zinc, QM7b, ModelNet
from torch_geometric.utils import (
    to_networkx,
    homophily,
    erdos_renyi_graph,
    stochastic_blockmodel_graph,
    scatter,
    loop,
)
from torch_geometric.nn import GINConv, global_mean_pool, global_add_pool
from torch_geometric.nn.aggr import (
    DeepSetsAggregation,
)  # key to implement DeepSet input (batch_features, batch_index)
from torch_geometric.nn.models import MLP
import torch.nn as nn
from torch.nn import Linear, Sequential, ReLU, Dropout
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch.optim import SGD
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

data_dir = "/export/canton/data/yma93/anydim_transferability/OI-DS/"

/cis/home/yma93/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [4]:
data = pickle.load(
    open(
        data_dir + "ModelNet_np_withLabels_pts=100.pkl",
        "rb",
    )  
)
train_x, train_y, val_x, val_y = data["train_pos"], data["train_y"], data["val_pos"], data["val_y"]

In [2]:
data_dir = "/export/canton/data/yma93/anydim_transferability/OI-DS/"
import pickle
file_path = data_dir + "LB_GW_test.pkl"
with open(file_path, "rb") as file:
    try:
        file = pickle.load(file)
    except Exception as e:
        raise RuntimeError(f"Error loading pickle file: {e}")


In [3]:
file

Array([0.1956029 , 0.45575535, 0.38426992, ..., 0.09898844, 0.1533244 ,
       0.26663452], dtype=float32)

In [8]:
# n_train = len(xtrain_s)
# n_test = len(xtest_s)
# bs = 40
dist_true_train = pickle.load(open(data_dir + "LB_GW_train.pkl", "rb"))
dist_true_train = torch.from_numpy(np.array(dist_true_train)).unsqueeze(1)
# dist_true_test = pickle.load(open(data_dir + "LB_GW_test.pkl", "rb"))
# dist_true_test = torch.from_numpy(np.array(dist_true_test)).unsqueeze(1)

RuntimeError: std::bad_cast

In [37]:
def get_fs(X):
    """
    input: X (n by 3)
    output: Data (class) storing fs (n by (n + n(n-1)/2 + 1)) feature matrix
     f_d is the set of diagonal edge attributes,
     f_o is the set of off-diagonal (upper) edge attributes,
     f_star is \sum_{i \neq j} X_ii X_ij where X is the n by n edge attribute graph
    """
    n = X.shape[0]
    data = Data()
    Gram = torch.FloatTensor(X @ X.T)
    off_mask = torch.triu(torch.ones(n, n)) == 1

    data.f_d = torch.diagonal(Gram, 0).unsqueeze(1)
    data.f_o = Gram[off_mask].unsqueeze(1)
    data.f_star = (
        (data.f_d * (torch.sum(Gram, dim=1, keepdim=True) - data.f_d)).sum().reshape((1, 1))
    )

    return data


def get_Kmeans(X, k=3):
    """
    Implement eqn 6: replace f_d as the self-dots of the K-means dots
    input: X (n by 3), k: number of k-means clusters
    output: Data (class) storing  feature matrix
     f_d is the flattened feature of the gram matrix of K-means centroids
     f_o is the set of dot products of each point to the K-means centroids
    """
    n = X.shape[0]
    data = Data()

    kmeans = KMeans(n_clusters=k, random_state=0, init="k-means++", n_init=1).fit(X)
    K = kmeans.cluster_centers_  # (num_centroids, 3)
    # sort Kmeans centroid by norm
    indexlist = np.argsort(np.linalg.norm(K, axis=1))
    K = K[indexlist, :]
    data.f_o = torch.FloatTensor(X @ K.T)  # shape(n, d)
    Gram_k = torch.FloatTensor(K @ K.T)  # shape (d, d)
    data.f_d = Gram_k.reshape(1, k * k)  # shape (d^2,)
    return data

In [ ]:
def build_dataset(xtrain_s, xtest_s, ids, Kmeans=False, k=3):
    X_train = []
    X_test = []
    for i in ids:
        if Kmeans:
            X_train.append(get_Kmeans(xtrain_s[i], k=k))
            X_test.append(get_Kmeans(xtest_s[i], k=k))
        else:
            X_train.append(get_fs(xtrain_s[i]))
            X_test.append(get_fs(xtest_s[i]))

    return X_train, X_test


def get_loaders(train_data1, test_data1, train_data2, test_data2, batch_size, follow_batch=None):
    """
    data1, data2 refers to subsets in ModelNet10 belonging to two different classes
    """

    train_loader1 = DataLoader(
        train_data1, batch_size=batch_size, shuffle=False, follow_batch=follow_batch
    )
    train_loader2 = DataLoader(
        train_data2, batch_size=batch_size, shuffle=False, follow_batch=follow_batch
    )

    test_loader1 = DataLoader(
        test_data1, batch_size=batch_size, shuffle=False, follow_batch=follow_batch
    )

    test_loader2 = DataLoader(
        test_data2, batch_size=batch_size, shuffle=False, follow_batch=follow_batch
    )

    return train_loader1, train_loader2, test_loader1, test_loader2